In [ ]:
# Fully run via Kaggle: https://www.kaggle.com/code/lilfatdog/metplanet-3350-short-positions-daily-chart

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import plotly.graph_objects as go

from datetime import datetime, timedelta

In [ ]:
# Column names as reference for anyone who wants to play around and graph different things based on the dataset
# schema = [
#     "date_of_calculation", "code_of_stock", "name_of_stock", "name_of_short_seller", "address_of_short_seller",
#     "name_of_discretionary_investment_contractor", "address_of_discretionary_investment_contractor", "name_of_investment_fund", "ratio_of_short_positions_to_shares_outstanding",
#     "number_of_short_positions_in_shares", "number_of_short_positions_in_trading_units", "date_of_calculation_in_previous_reporting", "ratio_of_short_positions_in_previous_reporting", "notes"
# ]
LOOKBACK_WINDOW = 29 # view past 30-day short seller positions

### LOAD DATASET
df = pd.read_csv('/kaggle/input/metaplanet-daily-short-positions/metaplanet_daily_short_positions.csv')
df = df[['date_of_calculation', 'name_of_short_seller', 'number_of_short_positions_in_shares']]
# Rename MUFG and take out the Kanji
df.loc[df['name_of_short_seller'].str.contains('MUFG', na=False), 'name_of_short_seller'] = 'MUFG'
df['date_of_calculation'] = pd.to_datetime(df['date_of_calculation'])
# print(df)


### CONFIGURABLE: chart will depend on the 'LOOKBACK_WINDOW' var, I just chose p30D window
end_date = pd.to_datetime(datetime.today().strftime('%Y-%m-%d'))
start_date = end_date - pd.Timedelta(days=LOOKBACK_WINDOW)
date_range = pd.date_range(start=start_date, end=end_date)
# print(date_range)


# Get last known position for each seller before the lookback window
# Note: the daily JPX data won't have an entry to a Short Seller if said SS doesn't update their position (i.e.: increase/decrease/close out their position)
# So to make sure we "best-effort" include all short positions, we scan the latest entry outside the lookback window in case a SS opened a position long ago and haven't moved.
before_window = df[df['date_of_calculation'] < start_date]
last_known_position = before_window.sort_values('date_of_calculation').groupby('name_of_short_seller').last()
# print(last_known_position)


# Data within the window
in_window = df[df['date_of_calculation'] >= start_date]
# print(in_window)

# Combine last known (as a "start_date" row) with in-window data
frames = []
for seller in df['name_of_short_seller'].unique():
    seller_data = in_window[in_window['name_of_short_seller'] == seller].copy()
    if seller in last_known_position.index:
        # Add a row at start_date with the last known value before window
        row = pd.DataFrame({
            'date_of_calculation': [start_date],
            'name_of_short_seller': [seller],
            'number_of_short_positions_in_shares': [last_known_position.loc[seller, 'number_of_short_positions_in_shares']]
        })
        seller_data = pd.concat([row, seller_data])
    frames.append(seller_data)

full_window = pd.concat(frames)
# print(full_window)

In [ ]:
pivot = full_window.pivot_table(
    index='date_of_calculation',
    columns='name_of_short_seller',
    values='number_of_short_positions_in_shares',
    aggfunc='last'
)
pivot = pivot.reindex(date_range)
pivot = pivot.ffill()

fig = go.Figure()

seller_order = pivot.max().sort_values(ascending=False).index
for seller in seller_order:
    fig.add_trace(go.Scatter(
        x=pivot.index,
        y=pivot[seller],
        mode='lines+markers',
        name=seller
    ))

fig.update_layout(
    title=f'Short Positions in Shares Over Last {LOOKBACK_WINDOW + 1} Days',
    xaxis_title='Date',
    yaxis_title='Number of Short Positions (Millions)',
    legend_title='Short Seller',
    hovermode='x unified',
    template='plotly_white',
    width=1000,
    height=600,
    annotations=[
        dict(
            text="Tip: Double-click a legend item to highlight one seller. Double-click again to show all. You can also select a subset of Short Sellers.",
            xref="paper", yref="paper",
            x=0, y=1.10, showarrow=False, font=dict(size=12, color="gray")
        )
    ]
)
fig.show()

In [ ]:
# Matplotlib static chart for output mode
colors = plt.get_cmap('tab20').colors
ax = pivot.plot(figsize=(12,6), marker='o', color=colors)
plt.title(f'Short Positions in Shares Over Last {LOOKBACK_WINDOW + 1} Days')
plt.xlabel('Date')
plt.ylabel('Number of Short Positions in Shares (Millions)')
plt.legend(title='Short Seller', bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.grid(True)

ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{x*1e-6:.0f}M'))

plt.tight_layout()
plt.show()